In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rawankhaled20","key":"f5a17aecb67ca4675b425dadbe4ab60e"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                        title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/top-spotify-songs-2023                    Most Streamed Spotify Songs 2023                 47KB  2023-08-26 11:04:57          17802        560  1.0              
joebeachcapital/students-performance                       Students Performance                              2KB  2023-08-31 00:50:11           6355        160  1.0              
mexwell/motorbike-marketplace                              🏍️ Motorbike Marketplace                          2MB  2023-09-18 05:51:56           1019         27  1.0              
iamsouravbanerjee/airline-dataset                          Airline Dataset                               

In [ ]:
!kaggle datasets download -d ahsanaseer/top-rated-tmdb-movies-10k

 70% 1.00M/1.43M [00:00<00:00, 2.00MB/s]
100% 1.43M/1.43M [00:00<00:00, 2.65MB/s]


In [ ]:
! unzip top-rated-tmdb-movies-10k.zip

Archive:  top-rated-tmdb-movies-10k.zip
  inflating: top10K-TMDB-movies.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
Dataset=pd.read_csv("top10K-TMDB-movies.csv")

In [ ]:
Dataset

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
...,...,...,...,...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,13995,Captain America,"Action,Science Fiction,War",en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668


In [ ]:
#Remove NAN values
Dataset.dropna(inplace=True)

In [ ]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9985 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9985 non-null   int64  
 1   title              9985 non-null   object 
 2   genre              9985 non-null   object 
 3   original_language  9985 non-null   object 
 4   overview           9985 non-null   object 
 5   popularity         9985 non-null   float64
 6   release_date       9985 non-null   object 
 7   vote_average       9985 non-null   float64
 8   vote_count         9985 non-null   int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 780.1+ KB


In [ ]:
# Not  very important features
Dataset.drop("original_language", axis=1, inplace=True)
Dataset.drop("release_date", axis=1, inplace=True)
# Note that we can make the following a catagory of selection but for now will stick on the genre and overview and popularity
Dataset.drop("vote_average", axis=1, inplace=True)
Dataset.drop("vote_count", axis=1, inplace=True)
movies=Dataset

In [ ]:
#convert text in Dataset to vectors

In [ ]:
CV=CountVectorizer(max_features=10000, stop_words='english')

In [ ]:
genre_vector=CV.fit_transform(movies['genre'].values.astype('U')).toarray()
overview_vector=CV.fit_transform(movies['overview'].values.astype('U')).toarray()
popularity_vector=CV.fit_transform(movies['popularity'].values.astype('U')).toarray()

In [ ]:
# You can Calculate the similarity based on two approches wheather the overview or the genre
sim1=cosine_similarity(genre_vector)
sim2=cosine_similarity(overview_vector)
sim3=cosine_similarity(popularity_vector)

In [ ]:
# Sorting sim1[2] based on genre_vector[1]
dist = sorted(list(enumerate(sim1[2])), reverse=True, key=lambda vector: vector[1])
for i in dist[1:6]:
    print(i)
    print(movies.iloc[i[0]].title)
print()
# Sorting sim2[2] based on overview_vector[1]
dist2 = sorted(list(enumerate(sim2[2])), reverse=True, key=lambda vector: vector[1])
for i in dist2[1:6]:  # Use dist2 here, not dist
    print(i)
    print(movies.iloc[i[0]].title)

(2, 0.9999999999999998)
The Godfather
(4, 0.9999999999999998)
The Godfather: Part II
(23, 0.9999999999999998)
GoodFellas
(26, 0.9999999999999998)
Once Upon a Time in America
(36, 0.9999999999999998)
City of God

(4, 0.3955774026426199)
The Godfather: Part II
(892, 0.24720661623652207)
Kind Hearts and Coronets
(3926, 0.23570226039551587)
Sinister
(747, 0.2286647801900117)
Still Walking
(250, 0.22360679774997894)
On My Skin


In [ ]:
def recommend_movie(movie,sim):
  index=movies[movies['title']==movie].index[0]
  dist=sorted(list(enumerate(sim[index])), reverse=True, key=lambda  vector:vector[1])
  for i in dist[1:6]:
    print(i)
    print(movies.iloc[i[0]].title)

In [ ]:
rec=input("what movie do you want me to recommend: ")
print("what catagory do you suggest me to recommend based on")
print(" 1.genre\n 2.overview\n 3.popularity")
cato=int(input("Choose a number: "))

what movie do you want me to recommend: Iron Man
what catagory do you suggest me to recommend based on
 1.genre
 2.overview
 3.popularity
Choose a number: 3


In [ ]:
if cato == 1:
  sim = sim1
elif cato == 2:
  sim = sim2
elif cato == 3:
  sim = sim3
else:
    print("Invalid choice")
    sim = None

if sim is not None:
    if rec in movies['title'].values:
        recommend_movie(rec, sim)
    else:
        print(f"Movie '{rec}' not found in the dataset.")


(1085, 0.7071067811865475)
Hannah and Her Sisters
(1754, 0.7071067811865475)
Sorry We Missed You
(2259, 0.7071067811865475)
Great Expectations
(4730, 0.7071067811865475)
Eye for an Eye
(5298, 0.7071067811865475)
Night Train to Lisbon
